In [43]:
# Importamos librerías
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [44]:
# El usuario introduce la palabra a buscar 
buscar = input("¿Qué deseas descargar?")
driver = webdriver.Chrome()

¿Qué deseas descargar?cat


In [45]:
# Se busca en imageNet el objeto
driver.get(f"http://image-net.org/search?q={buscar}")
# Se obtiene el path para extraer el ID con la herramienta inspeccionar elementos
elements = driver.find_elements_by_tag_name('table tbody tr td table tbody tr td a')
setSynsets = set([])
# Se obtiene solo el ID
for e in elements:
    url = e.get_attribute('href')
    if (len(url) == 42):
        setSynsets.add(url[-9:])

{'n02124075', 'n02123159', 'n02123394', 'n02509815', 'n02497673', 'n02123045', 'n02123597'}


In [46]:
import numpy as np
import urllib.request
import cv2
#Con el url se descarga la imagen correspondiente. 
def url_to_image(url):
    try:
        resp = urllib.request.urlopen(url)
        image = np.asarray(bytearray(resp.read()), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        return image
    except:
        return None

In [47]:
imagenes = []
# Maxima cantidad de imagenes a descargar.
max_imag = 100

for synset in setSynsets:
    driver.get(f"http://www.image-net.org/api/text/imagenet.synset.geturls?wnid={synset}")
    links = driver.find_element_by_tag_name('pre').text.split('\n')
    imagenes.extend(links)
imagenes_conv = []
cont = 0

# Por cada imagen encontrada se descarga.
for imagen in imagenes:
    conv = url_to_image(imagen)
    # Se pudo obtener la imagen correctamente.
    if (conv is not None):
        imagenes_conv.append(conv)
        cont = cont + 1
    # Se llegó al maximo de imagenes a descargar.
    if (cont == max_imag):
        break


10673
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
100


In [48]:
import os
# Se obtine el path actual y se crea uno para train y test.
path = os.getcwd()
train = os.path.join(path,'train',buscar)
test = os.path.join(path,'test',buscar)

# crear directorios si no existen.
if not os.path.isdir(os.path.join(path,'test')):
    os.mkdir(os.path.join(path,'test'))
    os.mkdir(test)
if not os.path.isdir(os.path.join(path,'train')):
    os.mkdir(os.path.join(path,'train'))
    os.mkdir(train)

# Se guardan las imagenes en su carpeta correspondiente.
cantidad_train = round(len(imagenes_conv) * .80)
index = 0
for imagen in imagenes_conv:
    if (cantidad_train > index):
        cv2.imwrite(os.path.join(train,f'train_{index}.jpg'), imagen)
    else:
        cv2.imwrite(os.path.join(test,f'test_{index}.jpg'), imagen)
    index = index+1
